In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory as img_from_dir
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def PSNR(y_true,y_pred):
  mse=tf.reduce_mean( (y_true - y_pred) ** 2 ).numpy()
  return 20 * log10(1 / (mse ** 0.5))

def log10(x):
  numerator = tf.math.log(x)
  denominator = tf.math.log(tf.constant(10, dtype=numerator.dtype)).numpy()
  return numerator / denominator

def pixel_MSE(y_true,y_pred):
  return tf.reduce_mean( (y_true - y_pred) ** 2 ).numpy()


In [ ]:
path="/path/"
imgs_path="/sample/"
test_data=img_from_dir(imgs_path,label_mode=None,image_size=(128,128), 
                                 validation_split=0.1,seed=70)


In [ ]:
@tf.function
def build_data(data):
  cropped=tf.dtypes.cast(tf.image.random_crop(data[0] / 255,(128,128,3)),tf.float32)

  lr=tf.image.resize(cropped,(32,32))
  #lr=tf.image.resize(lr,(128,128),method=tf.image.ResizeMethod.BICUBIC)
  return (lr,cropped)

def bicubic_interpolate(image,shape):
  img_resized=cv2.resize(image,shape, interpolation=cv2.INTER_CUBIC)
  return img_resized

val_dataset_mapped = test_data.map(build_data,num_parallel_calls=tf.data.AUTOTUNE)
SRResnet_old=load_model(f'models/SRResNet_gen_latest.h5')
SRResnet_latest=load_model(f'models/SRResNet_gen_latest.h5') #compare
psnr_tot, psnr_tot1, MSE, MSE1=[], [], [], []
low_res, original, predictions1, predictions2=[], [], [], []

for i in val_dataset_mapped.take(1000):
    im=i[1].numpy()
    original.append(im)
    lr=bicubic_interpolate(im,(im.shape[1]//4,im.shape[0]//4))
    pred=SRResnet_old(np.array([lr]))
    pred1=SRResnet_latest(np.array([lr]))
    low_res.append(lr)
    predictions1.append(pred)
    predictions2.append(pred1)
    psnr=PSNR(im,pred[0].numpy())
    psnr1=PSNR(im,pred1[0].numpy())
    mse=pixel_MSE(im,pred[0].numpy())
    mse1=pixel_MSE(im,pred1[0].numpy())
    MSE.append(mse)
    MSE1.append(mse1)
    psnr_tot.append(psnr)
    psnr_tot1.append(psnr1)

In [ ]:
print(f"PSNR:\nOLD:{np.mean(psnr_tot1)};     Latest:{np.mean(psnr_tot)}")
print(f"MSE:\nOLD:{np.mean(MSE1)};    Latest:{np.mean(MSE)}")

In [ ]:
for i in range(15):
    pred
    fig=plt.figure(figsize=(20,5))
    plt.subplot(1,4,1)
    plt.title("Low Resolution Image")
    plt.imshow(lr)
    plt.axis('off')
    plt.subplot(1,4,2)
    plt.title("Super-Resolution Image(first model)\nwith 10k imgs & 50 epochs")
    plt.imshow(predictions1[i][0])
    plt.axis('off')
    plt.subplot(1,4,3)
    plt.title("Super-Resolution Image(second model)\n5k imgs & 100 epochs")
    plt.imshow(predictions2[i][0])
    plt.axis('off')
    plt.subplot(1,4,4)
    plt.title("Original Image")
    plt.imshow(im)
    plt.axis('off')
    plt.show()

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(2,2,1)
plt.plot(psnr_tot)
plt.subplot(2,2,2)
plt.plot(psnr_tot1)
plt.subplot(2,2,3)
plt.plot(MSE)
plt.subplot(2,2,4)
plt.plot(MSE1)
plt.show()